# SUBSIDIES
This is a scraper for subsidy applications in a platform [QOLDAU](https://subsidies.qoldau.kz/ru/subsidies/).  
There're 3 categories:
  - Pesticides(since 2020, before was only herbicides)
  - Seeds
  - Fertilizers

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# LIBRARIES
# *********
# request for the web page
import requests

# handle while requesting for the web page
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# create a simulation of the pseudo-user
from selenium import webdriver

# set sleep on web page on every click, handling nginx error
import time
import datetime

# to get web page soure
from bs4 import BeautifulSoup

# create df
import pandas as pd 

# manipulate with shape of the df
import numpy as np

# record start time
from time import gmtime, strftime

# get rid off quotes
import csv

# CONNECT DB
import pyodbc

In [3]:
conn = pyodbc.connect('Driver=SQL Server;Server=192.168.1.209;Database=AA_DWH_X;Trusted_Connection=yes;')
cursor = conn.cursor()

### Links

In [50]:
# baseURL of Qoldau
baseUrl = "https://subsidies.qoldau.kz/ru/subsidies/"

# applications
appFerti = "fertilizer/pub-apps"
appPest = "herbicides/pub-apps"
appSeed = "seed/pub-apps"

# appFerti_2019 = "fertilizer/pub-apps?Year=2019"
# appPest_2019 = "herbicides/pub-apps?Year=2019"

# dicts of prods
# sprPest = "herbicides/refs/usage-norms"
# sprSeed = "seed/seeds"

listOfLinks = [appFerti, appPest, appSeed]#, appPest_2019 , appFerti_2019, sprSeed] #  , sprSeed, sprPest  
listOfNames =  ["Удобрения", "Пестициды", "Семена"]#, "Пестициды2019", "Удобрения2019", "СправочникСемян"] #,  , 'спрСемена', 'спрПестициды' 

### Memory usage counter

In [51]:
import tracemalloc
tracemalloc.start()
# my_complex_analysis_method()
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
tracemalloc.stop()

Current memory usage is 29.894412MB; Peak was 32.057666MB


### Scraper

In [52]:
tracemalloc.start() ##############################

options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
d = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Program Files (x86)\chromedriver.exe')
baseUrl = baseUrl
allLinks = listOfLinks 
names = listOfNames   
allData = []
linkN = 0
current, peak = tracemalloc.get_traced_memory() ##############################

# func insert DB
def insert_into (f):
    f["kategoria"]=names[linkN-1]
    f=f[['Область', 'Наименование заявителя', 'Дата подачи заявки', 'Статус', 'Площадь использования, га', 'Поставщик', 'Наименование препарата', 'Единица измерения', 'Объем', 'Сумма субсидий (расчетная), тг.', 'Тип заявки', 'Дата принятия заявки', 'Дата отклонения', 'Примечание к отрицательному решению', 'Причина отклонения', 'IP подачи заявки', 'IP отзыва заявки', 'IP принятия заявки', 'IP отклонения заявки','kategoria']]
    insertQuery = '''
    INSERT INTO [L0_X].[dbo].[SUBSIDIES_PARSER]
                (
               [REGION]
  ,[APPLICANT]
  ,[APPDATE]
  ,[STATUS]
  ,[USAGEAREA]
  ,[PROVIDER]
  ,[PRODUCT]
  ,[UNIT]
  ,[VOLUME]
  ,[SUBSIDIES]
  ,[APPTYPE]
  ,[APPACCEPTANCEDATE]
  ,[APPREJECTIONDATE]
  ,[NOTEONNEGATIVERESOLUTION]
  ,[REJECTIONREASON]
  ,[IP_APPLICANT]
  ,[IP_WITHDRAW]
  ,[IP_OF_ACCEPTANCE]
  ,[IP_OF_REJECTION ]
  ,[GROUP]
                )
         VALUES
               (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    # i = 0
    try:
        conn.autocommit = False
        for t in f.values.tolist():
    #         t.append(i)
    #         i+=1
            cursor.execute(insertQuery, t)
    except Exception as e:
        print(t)
        cursor.rollback()
        print(e)
        print('transaction rolled back')
    else:
        print('inserted successfully')
        conn.commit()
    finally:
        conn.autocommit = True
    

for link in allLinks:
    print(linkN, " = ", names[linkN])
    linkN+=1
    print("Started: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
    d.get(baseUrl+link)
    time.sleep(2)
    html = d.page_source
    nPages = int(d.find_elements_by_xpath('//li[@data-page-index]/a')[-2].text)
    print("This page's Navgation steps = ", nPages)
    tempLastN = []
    tempRowData = []
    cols = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table").find_elements_by_tag_name('th')
    colNames = []
    tempColNames = []
    for col in cols:
        r = col.get_attribute("innerText").replace('\n', ',')
        time.sleep(1)
        tempColNames.append(r)
    colN = len(tempColNames)
#     print(colN)
#     print(tempColNames)
    for i in range(len(tempColNames)):
        if tempColNames[i] in ['Производитель пестицидов','Поставщик семян','Производитель удобрений']:
            tempColNames[i]="Поставщик"
        elif tempColNames[i] in ["Наименование препарата","Наименование","Наименование удобрения"]:
            tempColNames[i]="Наименование препарата"
        elif tempColNames[i] in ["Объем пестицида в базовой ед.измерения","Объем семян в базовой ед.измерения","Объем удобрения"]:
            tempColNames[i]="Объем"
        elif tempColNames[i] in ["Базовая единица измерения","Базовая единица измерения"]:
            tempColNames[i]="Единица измерения"
        elif tempColNames[i]=="Дата отрицательного решения":
            tempColNames[i]="Дата отклонения"
        elif tempColNames[i]=="Причина отрицательного решения":
            tempColNames[i]="Причина отклонения"
            
    print(tempColNames)
#     check RECORDINGS 
    RECORDINGS=d.find_element_by_xpath("//span[@class='badge badge-secondary-lighten']").text
    RECORDINGS=RECORDINGS.split("из ")
    RECORDINGS=int(RECORDINGS[1])
    print(RECORDINGS)
#     for LastN in d.find_elements_by_xpath("//div[@class='pull-right']/div/span"):
#         tempLastN.append(LastN.text.split(' '))
#     lastN = int(tempLastN[-1][-1])



    
    for click in range(nPages):
        print(click)
        time.sleep(2)
        rows = d.find_element_by_xpath("//div[@class='sw-result-table-container']/div/div/table/tbody").find_elements_by_tag_name('td')
        if(rows[0].text != "Нет записей"):
#             RECORDINGS_click=[]
            for row in rows:
                tempRowData.append(row.text)
            RECORDINGS_click=d.find_element_by_xpath("//span[@class='badge badge-secondary-lighten']").text #количество записи
            RECORDINGS_click=RECORDINGS_click.split("из ")
            RECORDINGS_click_int=int(RECORDINGS_click[1])
            if(click==nPages-1): 
                d.find_elements_by_xpath('//li[@data-page-index]/a')[-2].click()
            else:
                d.find_elements_by_xpath('//li[@data-page-index]/a')[-1].click()
            html = d.page_source
            s = BeautifulSoup(html, 'html.parser')
        count = 0
        testData = np.reshape(tempRowData, (-1, colN))
        f = pd.DataFrame(testData, columns=tempColNames)
        f=f.drop('№',axis=1)
        if (RECORDINGS_click_int!=RECORDINGS):              #количество записи новый
            difference=RECORDINGS_click_int-RECORDINGS     #количество записи новый - количество записи 
            print(RECORDINGS_click_int)
            f=f.drop(labels = [0,difference],axis = 0)
            insert_into(f)
            RECORDINGS=RECORDINGS_click_int    #обнова количество записи
            testData=[]
            f = f[0:0] 
            tempRowData=[]
        else:
            insert_into(f)
            testData=[]
            f = f[0:0] 
            tempRowData=[]
            
#     print(testData[0])    
#         print(testData)
    
    
#     f.to_csv(r'C:\Users\Admin\Documents\qldData\/' + names[linkN] + '.csv', encoding='utf-8', sep='|')#, quoting = csv.QUOTE_NONE, escapechar = '')
#     f.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\/' + names[linkN] + '.csv', encoding='utf-8', sep='|')#, quoting = csv.QUOTE_NONE, escapechar = '')
#     linkN += 1
#     tempRowData = []
    print("Finished: ", datetime.datetime.now().astimezone().strftime("%Y-%m-%d, %H:%M:%S"))
    print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

<ipython-input-52-0c36c2c730b3>:6: DeprecationWarning: use options instead of chrome_options
  d = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Program Files (x86)\chromedriver.exe')


0  =  Удобрения
Started:  2022-08-16, 18:02:10
This page's Navgation steps =  747
['№', 'Область', 'Наименование заявителя', 'Дата подачи заявки', 'Статус', 'Площадь использования, га', 'Поставщик', 'Наименование препарата', 'Единица измерения', 'Объем', 'Сумма субсидий (расчетная), тг.', 'Тип заявки', 'Дата принятия заявки', 'Дата отклонения', 'Примечание к отрицательному решению', 'Причина отклонения', 'IP подачи заявки', 'IP отзыва заявки', 'IP принятия заявки', 'IP отклонения заявки']
22396
0
inserted successfully
1
inserted successfully
2
inserted successfully
3
inserted successfully
4
inserted successfully
5
inserted successfully
6
inserted successfully
7
inserted successfully
8
inserted successfully
9
inserted successfully
10
inserted successfully
11
inserted successfully
12
inserted successfully
13
inserted successfully
14
inserted successfully
15
inserted successfully
16
inserted successfully
17
inserted successfully
18
inserted successfully
19
inserted successfully
20
inserte

inserted successfully
301
inserted successfully
302
inserted successfully
303
inserted successfully
304
inserted successfully
305
inserted successfully
306
inserted successfully
307
inserted successfully
308
inserted successfully
309
inserted successfully
310
inserted successfully
311
inserted successfully
312
inserted successfully
313
inserted successfully
314
inserted successfully
315
inserted successfully
316
inserted successfully
317
inserted successfully
318
inserted successfully
319
inserted successfully
320
inserted successfully
321
inserted successfully
322
inserted successfully
323
inserted successfully
324
inserted successfully
325
inserted successfully
326
inserted successfully
327
inserted successfully
328
inserted successfully
329
inserted successfully
330
inserted successfully
331
inserted successfully
332
inserted successfully
333
inserted successfully
334
inserted successfully
335
inserted successfully
336
inserted successfully
337
inserted successfully
338
inserted suc

inserted successfully
617
inserted successfully
618
inserted successfully
619
inserted successfully
620
inserted successfully
621
inserted successfully
622
inserted successfully
623
inserted successfully
624
inserted successfully
625
inserted successfully
626
inserted successfully
627
inserted successfully
628
inserted successfully
629
inserted successfully
630
inserted successfully
631
inserted successfully
632
inserted successfully
633
inserted successfully
634
inserted successfully
635
inserted successfully
636
inserted successfully
637
inserted successfully
638
inserted successfully
639
inserted successfully
640
inserted successfully
641
inserted successfully
642
inserted successfully
643
inserted successfully
644
inserted successfully
645
inserted successfully
646
inserted successfully
647
inserted successfully
648
22397
inserted successfully
649
inserted successfully
650
inserted successfully
651
inserted successfully
652
inserted successfully
653
inserted successfully
654
insert

inserted successfully
166
inserted successfully
167
inserted successfully
168
inserted successfully
169
inserted successfully
170
inserted successfully
171
inserted successfully
172
inserted successfully
173
inserted successfully
174
inserted successfully
175
inserted successfully
176
inserted successfully
177
inserted successfully
178
inserted successfully
179
inserted successfully
180
inserted successfully
181
inserted successfully
182
inserted successfully
183
inserted successfully
184
inserted successfully
185
inserted successfully
186
inserted successfully
187
inserted successfully
188
inserted successfully
189
inserted successfully
190
inserted successfully
191
inserted successfully
192
inserted successfully
193
inserted successfully
194
inserted successfully
195
inserted successfully
196
33032
inserted successfully
197
inserted successfully
198
inserted successfully
199
inserted successfully
200
inserted successfully
201
inserted successfully
202
inserted successfully
203
insert

...

['Костанайская область', 'КХ "Анжела Б"', '22.07.2022 09:27', 'Отрицательное решение', '500,00', 'Товарищество с ограниченной ответственностью "Дегеш Агро ЛТД"', 'ХИТ, водный раствор', 'литр', '1 000,00', '1 059 370,00', 'Прямая', '', '23.07.2022 14:41', 'Согласно «Правил субсидирования повышения урожайности и качества продукции растениеводства», утвержденных приказом Министра сельского хозяйства Республики Казахстан от 30 марта 2020 года № 107 (далее- Правила), отказ в предоставлении субсидий принимается по следующим основаниям (пункт 12 «Правил»):\n1) установление недостоверности документов, представленных оригинатором, элитсемхозом, семхозом, реализатором, сельхозтоваропроизводителем (сельхозкооперативом) для получения субсидий, и (или) данных (сведений), содержащихся в них;\n2) несоответствие оригинатора, элитсемхоза, семхоза, реализатора, сельхозтоваропроизводителя и (или) представленных данных и сведений, необходимых для получения субсидий, установленных настоящими Правилами;\nОс

inserted successfully
472
33038
inserted successfully
473
inserted successfully
474
inserted successfully
475
inserted successfully
476
inserted successfully
477
inserted successfully
478
inserted successfully
479
inserted successfully
480
inserted successfully
481
inserted successfully
482
inserted successfully
483
inserted successfully
484
inserted successfully
485
inserted successfully
486
inserted successfully
487
inserted successfully
488
inserted successfully
489
inserted successfully
490
inserted successfully
491
inserted successfully
492
inserted successfully
493
inserted successfully
494
inserted successfully
495
inserted successfully
496
inserted successfully
497
inserted successfully
498
inserted successfully
499
inserted successfully
500
inserted successfully
501
inserted successfully
502
inserted successfully
503
inserted successfully
504
inserted successfully
505
inserted successfully
506
inserted successfully
507
inserted successfully
508
inserted successfully
509
insert

inserted successfully
786
inserted successfully
787
inserted successfully
788
inserted successfully
789
inserted successfully
790
inserted successfully
791
inserted successfully
792
inserted successfully
793
inserted successfully
794
inserted successfully
795
inserted successfully
796
inserted successfully
797
inserted successfully
798
inserted successfully
799
inserted successfully
800
inserted successfully
801
inserted successfully
802
inserted successfully
803
inserted successfully
804
inserted successfully
805
inserted successfully
806
inserted successfully
807
inserted successfully
808
inserted successfully
809
inserted successfully
810
inserted successfully
811
inserted successfully
812
inserted successfully
813
inserted successfully
814
inserted successfully
815
inserted successfully
816
inserted successfully
817
inserted successfully
818
inserted successfully
819
inserted successfully
820
inserted successfully
821
inserted successfully
822
inserted successfully
823
inserted suc

inserted successfully
1098
inserted successfully
1099
inserted successfully
1100
inserted successfully
Finished:  2022-08-17, 02:25:36
Current memory usage is 0.193036MB; Peak was 0.203718MB
2  =  Семена
Started:  2022-08-17, 02:25:36
This page's Navgation steps =  316
['№', 'Область', 'Дата подачи заявки', 'Наименование заявителя', 'Поставщик', 'Наименование препарата', 'Объем', 'Единица измерения', 'Площадь использования, га', 'Статус', 'Тип заявки', 'Дата принятия заявки', 'Дата отклонения', 'Сумма субсидий (расчетная), тг.', 'Примечание к отрицательному решению', 'Причина отклонения', 'IP подачи заявки', 'IP отзыва заявки', 'IP принятия заявки', 'IP отклонения заявки']
9476
0
inserted successfully
1
inserted successfully
2
inserted successfully
3
inserted successfully
4
inserted successfully
5
inserted successfully
6
inserted successfully
7
inserted successfully
8
inserted successfully
9
inserted successfully
10
inserted successfully
11
inserted successfully
12
inserted successfull

inserted successfully
294
inserted successfully
295
inserted successfully
296
inserted successfully
297
inserted successfully
298
inserted successfully
299
inserted successfully
300
inserted successfully
301
inserted successfully
302
inserted successfully
303
inserted successfully
304
inserted successfully
305
inserted successfully
306
inserted successfully
307
inserted successfully
308
inserted successfully
309
inserted successfully
310
inserted successfully
311
inserted successfully
312
inserted successfully
313
inserted successfully
314
inserted successfully
315
inserted successfully
Finished:  2022-08-17, 03:46:12
Current memory usage is 0.193036MB; Peak was 0.203718MB


In [41]:
    print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 9.125177MB; Peak was 11.142845MB


In [60]:
page =['Костанайская область', 'КХ "Анжела Б"', '22.07.2022 09:27', 'Отрицательное решение', '500,00', 'Товарищество с ограниченной ответственностью "Дегеш Агро ЛТД"', 'ХИТ, водный раствор', 'литр', '1 000,00', '1 059 370,00', 'Прямая', '', '23.07.2022 14:41',
       'Согласно «Правил субсидирования повышения урожайности и качества продукции растениеводства», утвержденных приказом Министра сельского хозяйства Республики Казахстан от 30 марта 2020 года № 107 (далее- Правила), отказ в предоставлении субсидий принимается по следующим основаниям (пункт 12 «Правил»):\n1) установление недостоверности документов, представленных оригинатором, элитсемхозом, семхозом, реализатором, сельхозтоваропроизводителем (сельхозкооперативом) для получения субсидий, и (или) данных (сведений), содержащихся в них;\n2) несоответствие оригинатора, элитсемхоза, семхоза, реализатора, сельхозтоваропроизводителя и (или) представленных данных и сведений, необходимых для получения субсидий, установленных настоящими Правилами;\nОснованием для отказа в оказании государственной услуги, установленные законодательством Республики Казахстан является (п. 9 Стандарта оказания государственной услуги):\n1) установление недостоверности документов, представленных услугополучателем для получения государственной услуги, и (или) данных (сведений), содержащихся в них;\n2) несоответствие услугополучателя и (или) представленных данных и сведений, необходимых для оказания государственной услуги, установленных настоящими Правилами.\n\nВаша заявка отклонена в связи с несоответствием Правилам, а именно:\n\nПримечание: Согласно пункта 82 «Правил» расчет субсидий на пестициды рассчитываются с учетом регламента применения, указанного в Списке пестицидов (ядохимикатов) или норм расхода биоагента (энтомофага) на 1 гектар (грамм/гектар, штук/гектар) (с учетом научно-обоснованных норм применения, размещенных на веб-портале).\n\nГербицид ХИТ, водный раствор согласно «Список пестицидов, разрешенных к производству (формуляции), ввозу, хранению, транспортировке, реализации и применению на территории\nРеспублики Казахстан на 2013-2022 годы (Дополнение № 8)», утвержденного Приказом Председателя Комитета государственной инспекции в агропромышленном комплексе Министерства сельского хозяйства Республики Казахстан от мая 2021 года применяется на:\n1) Пары - Норма расхода- 1,5-2,5 л/га - Опрыскивание сорняков в период активного роста\n2) Поля предназначенные под посев разных с/х культур – Норма расхода- 1,5-2,0 л/га - Опрыскивание вегетирующих сорняков весной.'
       , 'несоответствие сельхозтоваропроизводителя (сельхозкооператива) и (или) представленных данных и сведений, необходимых для получения субсидий', '88.204.218.174', '', '', '88.204.218.214', 'Пестициды']
page

['Костанайская область',
 'КХ "Анжела Б"',
 '22.07.2022 09:27',
 'Отрицательное решение',
 '500,00',
 'Товарищество с ограниченной ответственностью "Дегеш Агро ЛТД"',
 'ХИТ, водный раствор',
 'литр',
 '1 000,00',
 '1 059 370,00',
 'Прямая',
 '',
 '23.07.2022 14:41',
 'Согласно «Правил субсидирования повышения урожайности и качества продукции растениеводства», утвержденных приказом Министра сельского хозяйства Республики Казахстан от 30 марта 2020 года № 107 (далее- Правила), отказ в предоставлении субсидий принимается по следующим основаниям (пункт 12 «Правил»):\n1) установление недостоверности документов, представленных оригинатором, элитсемхозом, семхозом, реализатором, сельхозтоваропроизводителем (сельхозкооперативом) для получения субсидий, и (или) данных (сведений), содержащихся в них;\n2) несоответствие оригинатора, элитсемхоза, семхоза, реализатора, сельхозтоваропроизводителя и (или) представленных данных и сведений, необходимых для получения субсидий, установленных настоящими П

In [58]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

r=session.get('https://subsidies.qoldau.kz')
if r.status_code == 200:
    print('Все в норме!')
 
if r.status_code == 404:
    print('Страница не существует!')

KeyboardInterrupt: 

In [49]:
# f=f.drop('№',1)
tempRowData

['1',
 'Кызылординская область',
 'Товарищество с ограниченной ответственностью "М.Шаменов шаруашылығы"',
 '10.08.2022 15:33',
 'Заявка подана',
 '35,00',
 'Товарищество с ограниченной ответственностью "KAZ Chemicals Trading House"',
 'Аммофос (N10, P 46)',
 'тонн',
 '5,00',
 '464 062,00',
 'Прямая',
 '',
 '',
 '',
 '',
 '185.250.30.221',
 '',
 '',
 '',
 '2',
 'Кызылординская область',
 'ШҚ "НАРИМАН"',
 '10.08.2022 13:51',
 'Заявка подана',
 '37,10',
 'Товарищество с ограниченной ответственностью "BaiDala"',
 'Сульфат аммония (N-21, S-24)',
 'тонн',
 '30,00',
 '1 090 976,00',
 'Прямая',
 '',
 '',
 '',
 '',
 '5.251.237.200',
 '',
 '',
 '',
 '3',
 'Кызылординская область',
 'КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО"ИКМАТ"',
 '10.08.2022 13:51',
 'Заявка подана',
 '235,52',
 'Акционерное общество "КазАзот"',
 'Селитра аммиачная марки Б (N-34,4)',
 'тонн',
 '65,20',
 '4 468 749,00',
 'Переводная',
 '',
 '',
 '',
 '',
 '95.82.83.166',
 '',
 '',
 '',
 '4',
 'Кызылординская область',
 'Нұрасыл ШҚ',
 '10.08.202

In [20]:

sev2020=f
for column in sev2020:
    print(column,"->", sev2020[column].astype(str).str.len().max(), sev2020[column].dtypes)
sev2020.sample(30)


Область -> 22 object
Наименование заявителя -> 60 object
Дата подачи заявки -> 16 object
Статус -> 32 object
Площадь использования, га -> 8 object
Производитель удобрений -> 84 object
Наименование удобрения -> 222 object
Единица измерения -> 4 object
Объем удобрения -> 9 object
Сумма субсидий (расчетная), тг. -> 12 object
Тип заявки -> 10 object
Дата принятия заявки -> 16 object
Дата отрицательного решения -> 0 object
Примечание к отрицательному решению -> 0 object
Причина отрицательного решения -> 0 object
IP подачи заявки -> 14 object
IP отзыва заявки -> 10 object
IP принятия заявки -> 15 object
IP отклонения заявки -> 0 object


ValueError: Cannot take a larger sample than population when 'replace=False'

In [31]:
insertQuery = '''
        INSERT INTO [L0_X].[dbo].[SUBSIDIES_FERTILIZER_2022]
                   ([REGION]
      ,[APPLICANT]
      ,[APPDATE]
      ,[STATUS]
      ,[USAGEAREA]
      ,[PROVIDER]
      ,[PRODUCT]
      ,[UNIT]
      ,[VOLUME]
      ,[SUBSIDIES]
      ,[APPTYPE]
      ,[APPACCEPTANCEDATE]
      ,[APPREJECTIONDATE]
      ,[NOTEONNEGATIVERESOLUTION]
      ,[REJECTIONREASON]
      ,[IP_APPLICANT]
      ,[IP_WITHDRAW]
      ,[IP_OF_ACCEPTANCE]
      ,[IP_OF_REJECTION ]
        )
             VALUES
                   (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        '''
        # i = 0
try:
    conn.autocommit = False
    for t in sev2020.values.tolist():
#         t.append(i)
#         i+=1
        cursor.execute(insertQuery, t)
except Exception as e:
#             print(t)
    cursor.rollback()
#             print(e)
    print('transaction rolled back')
else:
    print('inserted successfully')
    conn.commit()
finally:
    conn.autocommit = True

inserted successfully


In [6]:
sev2020

NameError: name 'sev2020' is not defined

### Altering csv files
need to replace path  
**`q.to_csv(r'C:\Users\M_MUKHANOVA\Desktop\Mini PowerBI\Satellite\qoldauData\q2020.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, escapechar = '')`**


In [ ]:
# baseColNames  = ['Область', 'ДатаПодачиЗаявки', 'Заявитель',
#                  'Поставщик', 'Продукт',
#                  'Объем', 'БазоваяЕдиницаИзмерения',
#                  'ПлощадьИспользованияГа', 'Статус', 'ТипЗаявки',
#                  'ДатаПринятияЗаявки', 'ДатаОтклонения',
#                  'СуммаСубсидийТг', 'ПричинаОтклонения',
#                  'ПримечаниеКОтрицательномуРешению', 'group', 'year']
      
      

baseColNames  = ['region', 'appDate', 'applicant',
                 'provider', 'product',
                 'volume', 'unit',
                 'areaOfUsage', 'status', 'typeOfApp',
                 'AppAcceptanceDate', 'DateOfRejection',
                 'subsidies', 'reasonOfRejection',
                 'NoteOnNegativeResolution', 'group', 'year']


#---------------------------------------------------------------------------------
baseColNamesReorderForFerti = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Производитель удобрений', 'Наименование удобрения', 
                        'Объем удобрения', 'Единица измерения',
                        'Площадь использования, га', 'Статус', 'Тип заявки',
                        'Дата принятия заявки', 'Дата отрицательного решения', 
                        'Сумма субсидий (расчетная), тг.', 'Причина отрицательного решения',
                        'Примечание к отрицательному решению', 'group', 'year']

#---------------------------------------------------------------------------------
baseColNamesReorderForSeeds = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Поставщик семян', 'Наименование', 'Объем семян в базовой ед.измерения',
                        'Базовая единица измерения', 'Площадь использования, га', 'Статус',
                        'Тип заявки', 'Дата принятия заявки', 'Дата отклонения',
                        'Сумма субсидий (расчетная), тг.', 'Причина отклонения',
                        'Примечание к отрицательному решению', 'group']

In [ ]:

#------------------------------------ P E S T I C I D E S ---------------------------------------
p20 =  pd.read_csv(r'C:\Users\Admin\Documents\qldData\Пестициды.csv', encoding='utf-8', sep='|')
p20['group'] = "Пестициды"
p20 = p20.iloc[:, 2:]
p20['year'] = "2020"
# print(p20.columns)
print(len(p20.columns))


p19 =  pd.read_csv(r'C:\Users\Admin\Documents\qldData\Пестициды2019.csv', encoding='utf-8', sep='|')
p19['group'] = "Пестициды"
p19 = p19.iloc[:, 2:]
p19['year'] = "2019"
# print(p19.columns)
print(len(p19.columns))


p = pd.concat([p19, p20], ignore_index = True)
p.columns = baseColNames

#------------------------------------ F E R T L I Z E R S ---------------------------------------

f20 =  pd.read_csv(r'C:\Users\Admin\Documents\qldData\Удобрения.csv', encoding='utf-8', sep='|')
f20['group'] = "Удобрения"
f20 = f20.iloc[:, 2:]
f20['year'] = "2020"

f19 =  pd.read_csv(r'C:\Users\Admin\Documents\qldData\Удобрения2019.csv', encoding='utf-8', sep='|')
f19['group'] = "Удобрения"
f19 = f19.iloc[:, 2:]
f19['year'] = "2019"

f = pd.concat([f19, f20], ignore_index = True)
f = f[baseColNamesReorderForFerti]
f.columns = baseColNames

#------------------------------------ S E E D S ------------------------------------------------
sAll =  pd.read_csv(r'C:\Users\Admin\Documents\qldData\Семена2019_2020.csv', encoding='utf-8', sep='|')
sAll['group'] = "Семена"
sAll = sAll.iloc[:, 2:]
sAll = sAll[baseColNamesReorderForSeeds]

s19 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2019", case = False)]
s19['year'] = "2019"
s20 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2020", case = False)]
s20['year'] = "2020"

s = pd.concat([s19, s20], ignore_index = True)
s.columns = baseColNames
#------------------------------------ A L L -- C A T E G O R I E S ------------------------------------------------
q = pd.concat([p, f, s], ignore_index = True)
q['applicantID'] =  q[['applicant', 'region']].agg('-'.join, axis=1)
q = q.replace(r'\n',' ', regex=True)

#------------------------------------ C O N S T R A C T I O N S ------------------------------------------------
colsToUpper = [0,2,3]
for colUpper in range(len(colsToUpper)):
    q.iloc[:, colsToUpper[colUpper]] = q.iloc[:, colsToUpper[colUpper]].str.upper()
q = q.replace(to_replace = ["ТОВАРИЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ", "ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ",
                            'ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ'], 
              value = "ТОО",
              regex=True)
q = q.replace(to_replace = ["КРЕСТЬЯНСКОЕ ХОЗЯЙСТВА", "КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО", "К/Х", "К\Х", "Ш/Қ", "Ш\Қ", 
                            "ШАРУА ҚОЖАЛЫҒЫ", "К.Х."], 
              value = "КХ",
              regex=True)
q = q.replace(to_replace = ["ФЕРМЕРСКОЕ ХОЗЯЙСТВО", "ФЕРМЕРСКОЕ ХОЗЯЙСТВО", "Ф/Х", "Ф\Х", "Ф/Қ", "Ф\Қ", 
                            "ФЕРМЕРЛІК ШАРУА ҚОЖАЛЫҒЫ", "Ф.Х."], 
              value = "ФХ",
              regex=True)
q = q.replace(to_replace = ["АКЦИОНЕРНОЕ ОБЩЕСТВО", "АКЦИОНЕРЛІК ҚОҒАМЫ"], 
              value = "АО",
              regex=True)
q = q.replace(to_replace = ["ӨНДІРІСТІК КООПЕРАТИВІ", "АУЫЛ ШАРУАШЫЛЫҒЫ ӨНДІРІСТІК КООПЕРАТИВІ",
                            "СЕЛЬСКОХОЗЯЙСТВЕННЫЙ ПРОИЗВОДСТВЕННЫЙ КООПЕРАТИВ", "ПРОИЗВОДСТВЕННЫЙ КООПЕРАТИВ"], 
              value = "СПК",
              regex=True)
q['applicant'] = q['applicant'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['provider'] = q['provider'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)

#------------------------------------ D I S T R I B U T O R S ------------------------------------------------
pak = ['ТОО ПОЛГАР АГРОКЕМ', 'ТОО ПЕРВАЯ АГРОХИМИЧЕСКАЯ КОМПАНИЯ']
q['provider'].replace({pak[0]: 'ПАК', pak[1]: 'ПАК', 'ТОО ALEM AGRO TRADE': 'ALEM AGRO'}, inplace=True, regex=True)
q['provider'] = q['provider'].str.replace('ТОО ALEM AGRO HOLDING АЛЕМАГРО ХОЛДИНГ', 'ALEM AGRO')
#------------------------------------  A A   l o g i c a l  c o l u m n ------------------------------------------------- 
q['АА'] = np.where(q['provider'] == 'ALEM AGRO', 'Да', 'Нет')
aaClient = q[['АА', 'applicantID']].loc[q['АА'] == "Да"].drop_duplicates().reset_index().iloc[:,2:]
q['isSubsAaClient'] = q.applicantID.isin(aaClient.applicantID).astype(bool)

#------------------------------------ P A T H  ------------------------------------
q.to_csv(r'C:\Users\Admin\Documents\qldData\q.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, escapechar = '')
#------------------------------------ R A N D O M   5   R O W S ------------------------------------
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
q.sample(5)

### Transpose & exporting final file to upload to the AA_DWH_X server

In [ ]:
test = pd.melt(q.reset_index(), id_vars = ['index']).sort_values(by = ['index'])
import csv
test.to_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\testQoldau.csv", header = ['index1', 'variable', 'value'], encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, escapechar = '')
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

### STOP tracemalloc

In [ ]:
tracemalloc.stop()

# Checking data

In [12]:
test1 = test
test1.columns = ['index1', 'cols', 'vals']
test

,index1,cols,vals
0,0,Область,АКТЮБИНСКАЯ ОБЛАСТЬ
481392,0,Поставщик,ТОО АКТОБЕ-АГРОСЕРВИС
641856,0,Продукт,САРАТОВСКАЯ 42
802320,0,Объем,"13 844,61"
962784,0,БазоваяЕдиницаИзмерения,кг
...,...,...,...
2406959,160463,ПримечаниеКОтрицательномуРешению,В соответствии подпункта 1) п. 12 Правил субси...
2567423,160463,Категория,Пестициды
2727887,160463,applicantID,КХ Отегенов Е.К.-Акмолинская область
1444175,160463,Статус,Отрицательное решение


In [10]:
test.shape

(3209280, 3)

In [15]:
test1 = test1.pivot_table(index=['index1'], columns='cols', aggfunc= lambda x: x)
test1.columns = test1.columns.droplevel().rename(None)
test1

,applicantID,АА,БазоваяЕдиницаИзмерения,ДатаОтклонения,ДатаПодачиЗаявки,ДатаПринятияЗаявки,Заявитель,Категория,Область,Объем,ПлощадьИспользованияГа,Поставщик,ПримечаниеКОтрицательномуРешению,ПричинаОтклонения,Продукт,Сезон,Статус,СуммаСубсидийТг,ТипЗаявки,клиентПолучившийСубсЧерезАА
index1,,,,,,,,,,,,,,,,,,,,
0,""""""" АКБАР """" КХ""-Актюбинская область",Нет,кг,NaN,28.11.2019 16:25,29.11.2019 10:40,АКБАР КХ,Семена,АКТЮБИНСКАЯ ОБЛАСТЬ,"13 844,61","1 064,97",ТОО АКТОБЕ-АГРОСЕРВИС,NaN,NaN,САРАТОВСКАЯ 42,Сезон 2019,Заявка оплачена,"1 176 791,00",Заявка на получение субсидий за фактически отп...,False
1,МИРЗАЛИЕВ КАРЖАУБАЙ ДОСИМОВИЧ-Туркестанская об...,Нет,кг,NaN,24.07.2019 10:56,NaN,МИРЗАЛИЕВ КАРЖАУБАЙ ДОСИМОВИЧ,Удобрения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,"4 350,00","8,70",ТОО КАЙНАР,NaN,NaN,Суперфосфат простой (P-15),Сезон 2019,Аннулирована,"82 650,00",Переводная,False
2,МИРЗАЛИЕВ КАРЖАУБАЙ ДОСИМОВИЧ-Туркестанская об...,Нет,кг,NaN,21.05.2019 11:47,NaN,МИРЗАЛИЕВ КАРЖАУБАЙ ДОСИМОВИЧ,Удобрения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,"2 610,00","8,70",ТОО КАЙНАР,NaN,NaN,Суперфосфат простой (P-15),Сезон 2019,Аннулирована,"61 987,50",Переводная,False
3,МИРЗАЛИЕВ КАРЖАУБАЙ ДОСИМОВИЧ-Туркестанская об...,Нет,кг,NaN,16.05.2019 11:15,28.05.2019 16:19,МИРЗАЛИЕВ КАРЖАУБАЙ ДОСИМОВИЧ,Удобрения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,"2 600,00","8,70",АО КАЗАЗОТ,NaN,NaN,"Селитра аммиачная (N 34,4)",Сезон 2019,Заявка оплачена,"70 948,50",Переводная,False
4,МИРЗАЛИЕВ НУРЛИБЕК ЖАСУЗАКОВИЧ-Туркестанская о...,Нет,тонн,NaN,06.08.2019 21:05,16.08.2019 16:00,МИРЗАЛИЕВ НУРЛИБЕК ЖАСУЗАКОВИЧ,Удобрения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,"8,10","20,05",АО КАЗАЗОТ,NaN,NaN,"Селитра аммиачная (N 34,4)",Сезон 2019,Заявка оплачена,"283 500,00",Переводная,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160459,"ТОО ""РЕАБВИ""-Акмолинская область",Нет,литр,NaN,09.07.2020 16:32,09.07.2020 18:54,ТОО РЕАБВИ,Пестициды,АКМОЛИНСКАЯ ОБЛАСТЬ,"200,00","49,00",ТОО ЕВРОГЕРБИЦИДЫ,NaN,NaN,"ТОРНАДО 540, водный раствор",Сезон 2020,Заявка оплачена,"95 599,00",Прямая,False
160460,"КХ ""АЛҒА""-Актюбинская область",Нет,литр,NaN,09.07.2020 16:32,NaN,КХ АЛҒА,Пестициды,АКТЮБИНСКАЯ ОБЛАСТЬ,"1 000,00","250,00",ТОО АКТОБЕ-АГРОСЕРВИС,NaN,NaN,"ТОРНАДО 540, водный раствор",Сезон 2020,Отозвана,"609 718,75",Прямая,False
160461,"ТОО ""ПРИРЕЧЕНСКОЕ""-Костанайская область",Нет,кг,NaN,09.07.2020 16:32,07.09.2020 18:31,ТОО ПРИРЕЧЕНСКОЕ,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"135,00","16 875,00",ТОО НЕРТУС АГРО,NaN,NaN,"СТРАЖ, водно-диспергируемые гранулы",Сезон 2020,Заявка оплачена,"949 218,75",Прямая,False


In [17]:
test1.columns

Index(['applicantID', 'АА', 'БазоваяЕдиницаИзмерения', 'ДатаОтклонения',
       'ДатаПодачиЗаявки', 'ДатаПринятияЗаявки', 'Заявитель', 'Категория',
       'Область', 'Объем', 'ПлощадьИспользованияГа', 'Поставщик',
       'ПримечаниеКОтрицательномуРешению', 'ПричинаОтклонения', 'Продукт',
       'Сезон', 'Статус', 'СуммаСубсидийТг', 'ТипЗаявки',
       'клиентПолучившийСубсЧерезАА'],
      dtype='object')

In [21]:
statuses = test1['Статус'].unique()
statuses 

array(['Заявка оплачена', 'Аннулирована', 'Отозвана',
       'Отрицательное решение', 'Не оплачена казначейством',
       'Заявка подана', 'Заявка у элитсемхоза', 'Сформирован платеж',
       'Заявка к оплате', 'Заявка у производителя удобрений',
       'Заявка у производителя СЗР'], dtype=object)

In [22]:
kost = test1[(test1['Область'] == 'КОСТАНАЙСКАЯ ОБЛАСТЬ') & (test1['Статус'] != 'Аннулирована') 
                                                          & (test1['Статус'] != 'Отозвана')
                                                          & (test1['Статус'] != 'Отрицательное решение') 
                                                          & (test1['Статус'] != 'Не оплачена казначейством')]
kost

,applicantID,АА,БазоваяЕдиницаИзмерения,ДатаОтклонения,ДатаПодачиЗаявки,ДатаПринятияЗаявки,Заявитель,Категория,Область,Объем,ПлощадьИспользованияГа,Поставщик,ПримечаниеКОтрицательномуРешению,ПричинаОтклонения,Продукт,Сезон,Статус,СуммаСубсидийТг,ТипЗаявки,клиентПолучившийСубсЧерезАА
index1,,,,,,,,,,,,,,,,,,,,
252,МИРЖИБАЕВ ЖАНИБЕК-Костанайская область,Нет,литр,NaN,19.06.2019 11:53,20.06.2019 19:03,МИРЖИБАЕВ ЖАНИБЕК,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"40,00","98,00",ТОО ПЕСТИЦИДЫ,NaN,NaN,"СПАЙДЕР, концентрат эмульсии",Сезон 2019,Заявка оплачена,"186 200,00",Прямая,False
257,МИРЖИБАЕВ ЖАНИБЕК-Костанайская область,Нет,литр,NaN,17.06.2019 12:59,18.06.2019 19:31,МИРЖИБАЕВ ЖАНИБЕК,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"30,00","98,00",ТОО ПЕСТИЦИДЫ,NaN,NaN,"ТРЕЛ 300, водный раствор",Сезон 2019,Заявка оплачена,"124 509,00",Прямая,False
1349,МУХАМЕТЖАНОВА СВЕТЛАНА-Костанайская область,Нет,литр,NaN,07.07.2019 14:44,07.08.2019 09:45,МУХАМЕТЖАНОВА СВЕТЛАНА,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"100,00","245,00",ТОО ПЕСТИЦИДЫ,NaN,NaN,"СПАЙДЕР, концентрат эмульсии",Сезон 2019,Заявка оплачена,"465 500,00",Прямая,False
1454,Михнев Александр Павлович-Костанайская область,Нет,литр,NaN,17.06.2019 22:51,19.06.2019 08:52,МИХНЕВ АЛЕКСАНДР ПАВЛОВИЧ,Удобрения,КОСТАНАЙСКАЯ ОБЛАСТЬ,"45,00","57,00",ТОО A.S.K. KAZAKHSTAN А.С.К. КАЗАХСТАН,NaN,NaN,Фертигрейн Старт (FERTIGRAIN START) (N-3%),Сезон 2019,Заявка оплачена,"68 102,68",Прямая,False
1463,Михнев Александр Павлович-Костанайская область,Нет,литр,NaN,17.06.2019 23:51,19.06.2019 08:54,МИХНЕВ АЛЕКСАНДР ПАВЛОВИЧ,Удобрения,КОСТАНАЙСКАЯ ОБЛАСТЬ,"40,00","332,00",ТОО A.S.K. KAZAKHSTAN А.С.К. КАЗАХСТАН,NaN,NaN,Текнофит РН (TECNOPHYT PH) (поли-гиброки-карбо...,Сезон 2019,Заявка оплачена,"55 625,00",Прямая,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160446,БАБИЧ ИВАН ИВАНОВИЧ-Костанайская область,Нет,литр,NaN,09.07.2020 16:17,07.09.2020 18:24,БАБИЧ ИВАН ИВАНОВИЧ,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"190,00","293,86",ТОО ЩЕЛКОВО АГРОХИМ-KZ,NaN,NaN,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии",Сезон 2020,Заявка оплачена,"497 562,50",Прямая,False
160449,"к/х ""Алькен С.Т.""-Костанайская область",Нет,литр,NaN,09.07.2020 16:23,07.09.2020 18:26,КХ АЛЬКЕН С.Т.,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"3 000,00","1 475,00",ТОО АСТАНА-НАН,NaN,NaN,"СМЕРЧ, водный раствор",Сезон 2020,Заявка оплачена,"2 048 400,00",Прямая,False
160451,"ТОО ""ТРОЯНА""-Костанайская область",Нет,литр,NaN,09.07.2020 16:26,07.09.2020 18:29,ТОО ТРОЯНА,Пестициды,КОСТАНАЙСКАЯ ОБЛАСТЬ,"500,00","837,00",ПАК,NaN,NaN,"ПРОГРЕСС, концентрат эмульсии",Сезон 2020,Заявка оплачена,"384 079,50",Прямая,True


In [49]:
kostSubs = kost[['СуммаСубсидийТг']].replace(',', '.', regex = True).replace(' ', '', regex = True)
mySum = kostSubs.astype(float).sum()
mySum

СуммаСубсидийТг    2.131157e+10
dtype: float64